In [1]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/')

In [4]:
libraries_list = ['/home/ananth/nsf_data_ingestion/libraries/pubmed_parser_lib.zip',
                  '/home/ananth/nsf_data_ingestion/libraries/unidecode_lib.zip']

In [2]:
import medline_parquet_write

In [3]:
medline_parquet_write.main()

INFO:root:Creating Spark Session....


Reading from /user/ananth/medline/medline_data/ and writing to /user/ananth/medline/parquet/.


INFO:root:Adding Libraries/home/ananth/nsf_data_ingestion/libraries/pubmed_parser_lib.zip
INFO:root:Adding Libraries/home/ananth/nsf_data_ingestion/libraries/unidecode_lib.zip


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3, ist-deacuna-n6.syr.edu, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/python/lib/pyspark.zip/pyspark/worker.py", line 217, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/python/lib/pyspark.zip/pyspark/worker.py", line 59, in read_command
    command = serializer._read_with_length(file)
  File "/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/python/lib/pyspark.zip/pyspark/serializers.py", line 170, in _read_with_length
    return self.loads(obj)
  File

In [6]:
spark.stop()

NameError: name 'spark' is not defined

In [ ]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/')
import zipfile
import os
import pyspark
from pyspark.sql import functions
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import Window
from pyspark.sql.functions import rank, max, sum, desc
import logging
from os import path
logging.getLogger().setLevel(logging.INFO)
findspark.init('/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/')
from pyspark.sql import SparkSession
libraries_list = ['/home/ananth/nsf_data_ingestion/libraries/pubmed_parser_lib.zip',
                  '/home/ananth/nsf_data_ingestion/libraries/config.zip',
                  '/home/ananth/nsf_data_ingestion/libraries/unidecode_lib.zip']

In [7]:
def package_imports():
    from config import nsf_config
    from config import spark_config
    from config.spark_config import exec_instance
    from config.spark_config import exec_mem
    from config.spark_config import exec_cores
    from config import data_source_params
    global config
    global nsf_config
    global spark_config
    global data_source_params

In [8]:
def read_hdfs_data(spark, param_list):
    data_path = param_list.get('xml_path')
    parquet_path = param_list.get('parquet_path')
    
    print("Reading from {} and writing to {}.".format(data_path, parquet_path))
    if(param_list.get('hdfs_read_type') == nsf_config.hdfs_read_BINARY):
        return spark.sparkContext.binaryFiles(os.path.join(data_path, '*.zip'), minPartitions=spark_config.minPartitions)
    elif(param_list.get('hdfs_read_type') == nsf_config.hdfs_read_WHOLEFILES):
        return spark.sparkContext.wholeTextFiles(os.path.join(data_path, '*.xml.gz'), minPartitions=spark_config.minPartitions)

In [9]:
def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    filepath = gzip_str[0]
    gzip_content = gzip_str[1]
    _, file_name = path.split(filepath)
    # decompress gzip

    xml_string = gzip_content.split("\n", 3)[3];
    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]



In [10]:
def create_session(libraries_list):
    logging.info('Creating Spark Session')
    spark = SparkSession.builder.getOrCreate()
    for library in libraries_list:
        logging.info('Adding Libraries' + str(library))
        spark.sparkContext.addPyFile(library)    # adding libraries
        spark.sparkContext.addPyFile(library)
        
    package_imports()    # adding libraries
    return spark

In [11]:
spark_session = create_session(libraries_list)

INFO:root:Creating Spark Session
INFO:root:Adding Libraries/home/ananth/nsf_data_ingestion/libraries/pubmed_parser_lib.zip
INFO:root:Adding Libraries/home/ananth/nsf_data_ingestion/libraries/config.zip
INFO:root:Adding Libraries/home/ananth/nsf_data_ingestion/libraries/unidecode_lib.zip


In [15]:
param_list = data_source_params.mapping.get('medline')
param_list

{'data_source_name': 'medline',
 'ftp_server': 'ftp.nlm.nih.gov',
 'medleasebaseline_url': '/nlmdata/.medleasebaseline/gz/',
 'medlease_url': '/nlmdata/.medlease/gz/',
 'hdfs_path': '/user/ananth/medline/',
 'xml_path': '/user/ananth/medline/medline_data',
 'parquet_path': '/user/ananth/medline/parquet/',
 'directory_path': '/home/ananth/airflow/medline_data/',
 'timestamp_file': 'time_stamp.txt',
 'hdfs_read_type': 'wholefiles'}

In [16]:
parquet_path = param_list.get('parquet_path')
parquet_path

'/user/ananth/medline/parquet/'

In [17]:

hdfs_data = read_hdfs_data(spark_session, param_list)


Reading from /user/ananth/medline/medline_data and writing to /user/ananth/medline/parquet/.


In [18]:
preprocess = hdfs_data.flatMap(parse_gzip_medline_str)


In [21]:
medline_df = preprocess.toDF()

In [22]:
medline_df.show()

+--------+-----------+-----------+-------------+------+--------------------+------------+-------+--------+--------------------+----------+-------------+--------+----------+--------+-------+--------------------+
|abstract|affiliation|     author|      country|delete|           file_name|issn_linking|journal|keywords|          medline_ta|mesh_terms|nlm_unique_id|other_id|       pmc|    pmid|pubdate|               title|
+--------+-----------+-----------+-------------+------+--------------------+------------+-------+--------+--------------------+----------+-------------+--------+----------+--------+-------+--------------------+
|        |           |JS Hildreth|United States| false|medline16n0001.xm...|   0065-9533|       |        |Trans Am Ophthalm...|          |      7506106|        |PMC1361495|16691646|   1865|Statement of Case...|
|        |           |BJ Jeffries|United States| false|medline16n0001.xm...|   0065-9533|       |        |Trans Am Ophthalm...|          |      7506106|    

In [1]:
from medline_parquet_write import main

In [2]:
main('medline')

INFO:root:Creating Spark Session
INFO:root:Adding Libraries/home/ananth/nsf_data_ingestion/libraries/pubmed_parser_lib.zip
INFO:root:Adding Libraries/home/ananth/nsf_data_ingestion/libraries/config.zip
INFO:root:Adding Libraries/home/ananth/nsf_data_ingestion/libraries/unidecode_lib.zip


Reading from /user/ananth/medline/medline_data and writing to /user/ananth/medline/parquet/.


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3, ist-deacuna-n6.syr.edu, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/python/lib/pyspark.zip/pyspark/worker.py", line 217, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/python/lib/pyspark.zip/pyspark/worker.py", line 59, in read_command
    command = serializer._read_with_length(file)
  File "/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/python/lib/pyspark.zip/pyspark/serializers.py", line 170, in _read_with_length
    return self.loads(obj)
  File

In [1]:
import sys
sys.path.append('/home/ananth/nsf_data_ingestion/')

In [4]:
from nsf_data_ingestion.federal_reporter import federal_parquet_write

In [5]:
federal_parquet_write.main('federal_reporter')

INFO:root:Creating Spark Session....
INFO:root:Writing to Parquet.....
INFO:root:Parquet Files Exist Deleting .......
INFO:root:Writing New parquet Files .......
INFO:root:Parquet Files Exist Deleting .......
INFO:root:Writing New parquet Files .......
INFO:root:Parquet Write Complete.....
